In [39]:
import pandas as pd
import numpy as np
from scipy import stats

In [58]:
results = pd.read_csv("Generalization results.csv")
methods = results.Method.unique()
methods = np.delete(methods, [2,-1])
methods = np.insert(methods, [0,2], ['Univariant', 'LSTM'])

In [59]:
methods

array(['Univariant', 'SVM_STI', 'FFN_STI', 'LSTM', 'DistilBERT_Mask',
       'DistilBERT_Partition', 'Roberta_Mask', 'Roberta_Partition'],
      dtype=object)

#### t-Test for Model V.S. Lexica

In [76]:
def t_test_modelvslexica(results):
    p = {}
    for i in range(len(methods)):
        if methods[i] not in p:
            p[methods[i]] = []
        data = results[results["Method"] == methods[i]]
        datawithin = data[data.Within == True]
        dataacross = data[data.Within == False]
        p[methods[i]].append(stats.ttest_rel(datawithin["modelAcc"],datawithin["lexiconAcc"]).pvalue)
        p[methods[i]].append(stats.ttest_rel(datawithin["modelF1"],datawithin["lexiconF1"]).pvalue)
        p[methods[i]].append(stats.ttest_rel(dataacross["modelAcc"],dataacross["lexiconAcc"]).pvalue)
        p[methods[i]].append(stats.ttest_rel(dataacross["modelF1"],dataacross["lexiconF1"]).pvalue)
    print(p)

In [77]:
# Sentiment
results_sentiment = results.copy()
results_sentiment = results_sentiment[(results_sentiment.TrainData=='yelp_subset')|(results_sentiment.TrainData=='amazon_toys_subset')|(results_sentiment.TrainData=='amazon_finefood_subset')|(results_sentiment.TrainData=='nrc_joy')]
t_test_modelvslexica(results_sentiment)

{'Univariant': [nan, nan, nan, nan], 'SVM_STI': [0.483536348156421, 0.4439706063629554, 0.18482340526398236, 0.32711848344560224], 'FFN_STI': [0.06452238268288872, 0.08939581647810328, 0.3048566780875831, 0.17273981730945034], 'LSTM': [0.02634960922789249, 0.0188141532910034, 0.00707622558267093, 0.0007452356279502401], 'DistilBERT_Mask': [0.016449213365099642, 0.013542611993293154, 5.4659316604774427e-14, 3.3807825982632305e-11], 'DistilBERT_Partition': [0.006439138543712099, 0.0038272212355000422, 5.605942117021118e-13, 2.0260173900466587e-10], 'Roberta_Mask': [0.012285540142678663, 0.0106169748599422, 3.766880353512135e-17, 1.0035959318837804e-14], 'Roberta_Partition': [0.004683999299350626, 0.0030404081180523354, 4.8797319029958e-13, 8.072221094433011e-11]}


In [78]:
# Emotion
results_emotion = results.copy()
results_emotion = results_emotion[(results_emotion.TrainData=='nrc_joy')|(results_emotion.TrainData=='nrc_surprise')|(results_emotion.TrainData=='nrc_sadness')|(results_emotion.TrainData=='nrc_fear')|(results_emotion.TrainData=='nrc_anger')]
results_emotion = results_emotion.drop(results_emotion[(results_emotion.TestData=='yelp_subset')|(results_emotion.TestData=='amazon_toys_subset')|(results_emotion.TestData=='amazon_finefood_subset')|(results_emotion.TestData=='emobank')].index)
t_test_modelvslexica(results_emotion)

{'Univariant': [nan, nan, nan, nan], 'SVM_STI': [0.028262693007284113, 0.024921827605154115, 0.0844516070959825, 0.3065791571053486], 'FFN_STI': [0.10116200262759636, 0.11433991514645175, 0.2928181895177969, 0.3830318545000383], 'LSTM': [0.01696843422797474, 0.013257600756537539, 0.005162388127066363, 0.017308004838682774], 'DistilBERT_Mask': [0.0005456490520456829, 0.002667047013928404, 0.0006955100080874829, 0.0003498791874349478], 'DistilBERT_Partition': [0.004006703268753135, 0.015075340917638697, 0.0056749271657337, 0.0033407116130359424], 'Roberta_Mask': [0.00030950468050543265, 0.0006551457189938323, 1.5788718383282994e-05, 8.108211046669808e-05], 'Roberta_Partition': [0.0007464200847908984, 0.0017341130816598573, 0.005387903185384761, 0.0022788885758447874]}


In [79]:
# All
t_test_modelvslexica(results)

{'Univariant': [nan, nan, nan, nan], 'SVM_STI': [0.05109532048718469, 0.04437589691108549, 0.03257409136066624, 0.14226044441105698], 'FFN_STI': [0.030528574689970275, 0.040294578327530896, 0.05658807911067907, 0.5483150726726036], 'LSTM': [0.008350403698850465, 0.005433631290250512, 0.00018820489183098493, 7.74265455500235e-05], 'DistilBERT_Mask': [8.66742173141599e-05, 9.854115061782467e-05, 5.91901090027749e-14, 3.9105099598676206e-13], 'DistilBERT_Partition': [5.5356881205732756e-05, 0.0005061847371431916, 1.6846759521145097e-11, 5.495693887076155e-11], 'Roberta_Mask': [2.4065614906026336e-05, 2.2181383340976627e-05, 1.94262837513625e-17, 1.741220656675822e-16], 'Roberta_Partition': [7.030222197569359e-06, 1.148420378965698e-05, 5.8110644306504645e-12, 7.476927384028798e-12]}


In [84]:
stats.ttest_ind(results_sentiment[results_sentiment.Method=='LSTM']['lexiconAcc'],results_emotion[results_emotion.Method=='LSTM']['lexiconAcc'])

Ttest_indResult(statistic=3.188945818951313, pvalue=0.0024650577850295115)

#### t-Test for Models and Lexicon Comparison

In [73]:
p = {}
results_indomain = results[results.Within==True]
results_across = results[results.Within==False]
for i in range(len(methods)):
    for j in range(i+1, len(methods)):
        pair = methods[i]+'-'+methods[j]
        if pair not in p:
            p[pair] = []
        data1_indomain = results_indomain[results_indomain["Method"]==methods[i]]
        data2_indomain = results_indomain[results_indomain["Method"]==methods[j]]
        data_indomain = data1_indomain.merge(data2_indomain, on=["TrainData", "TestData"])
        data1_across = results_across[results_across["Method"]==methods[i]]
        data2_across = results_across[results_across["Method"]==methods[j]]
        data_across = data1_across.merge(data2_across, on=["TrainData", "TestData"])
        for k in ["modelF1", "lexiconF1"]:
            p[pair].append(stats.ttest_rel(data_indomain[k+"_x"],data_indomain[k+"_y"]).pvalue)
            p[pair].append(stats.ttest_rel(data_across[k+"_x"],data_across[k+"_y"]).pvalue)
for i in range(4):
    for k in p:
        print(k, p[k][i])

Univariant-SVM_STI nan
Univariant-FFN_STI nan
Univariant-LSTM nan
Univariant-DistilBERT_Mask nan
Univariant-DistilBERT_Partition nan
Univariant-Roberta_Mask nan
Univariant-Roberta_Partition nan
SVM_STI-FFN_STI 0.5485945561365608
SVM_STI-LSTM 0.014432847154870287
SVM_STI-DistilBERT_Mask 0.001938154641953877
SVM_STI-DistilBERT_Partition 0.001938154641953877
SVM_STI-Roberta_Mask 3.6854235994048945e-05
SVM_STI-Roberta_Partition 3.6854235994048945e-05
FFN_STI-LSTM 0.01701878355647658
FFN_STI-DistilBERT_Mask 0.002591144891309217
FFN_STI-DistilBERT_Partition 0.002591144891309217
FFN_STI-Roberta_Mask 6.531937625472047e-05
FFN_STI-Roberta_Partition 6.531937625472047e-05
LSTM-DistilBERT_Mask 0.09509669494675396
LSTM-DistilBERT_Partition 0.09509669494675396
LSTM-Roberta_Mask 0.21977258588567028
LSTM-Roberta_Partition 0.21977258588567028
DistilBERT_Mask-DistilBERT_Partition nan
DistilBERT_Mask-Roberta_Mask 6.70223730681591e-05
DistilBERT_Mask-Roberta_Partition 6.70223730681591e-05
DistilBERT_Parti

In [61]:
data1_indomain = results_indomain[results_indomain["Method"]==methods[1]]
data2_indomain = results_indomain[results_indomain["Method"]==methods[0]]
data_indomain = data1_indomain.merge(data2_indomain, on=["TrainData", "TestData"])
data1_across = results_across[results_across["Method"]==methods[i]]
data2_across = results_across[results_across["Method"]==methods[j]]
data_across = data1_across.merge(data2_across, on=["TrainData", "TestData"])

In [64]:
stats.ttest_rel(data_indomain["lexiconF1_y"],data_indomain["lexiconF1_x"],)

Ttest_relResult(statistic=-5.0960939698068195, pvalue=0.0014056024196088027)

#### Lexicon Correlation

In [53]:
methods = {          
           'Univariant': 'uni',
           'FFN': 'ffn_feature',
           'SVM': 'svm_feature',
           'LSTM': 'lstm_attention',
           'Roberta_Mask': 'roberta_classification_mask', 
           'Roberta_Partition': 'roberta_classification_ps',
           'DistilBERT_Mask': 'distilbert_classification_mask',
           'DistilBERT_Partition': 'distilbert_classification_ps',
          }
methods_list = list(methods.keys())
datalist = ['amazon_finefood_subset', 'amazon_toys_subset', 'yelp_subset'] + ['nrc_'+i for i in ['surprise', 'joy', 'anger', 'sadness', 'fear']]

for i in range(len(methods_list)):
    for j in range(i+1, len(methods_list)):
        print(methods_list[i], methods_list[j])
        score = 0
        for data in datalist:
            lexica_1 = pd.read_csv('../lexica/'+methods_list[i]+'/'+data+'_'+methods[methods_list[i]]+'.csv')
            lexica_2 = pd.read_csv('../lexica/'+methods_list[j]+'/'+data+'_'+methods[methods_list[j]]+'.csv')
            for k in [lexica_1, lexica_2]:
                if 'word' in k.columns:
                    k.columns = ['Word', 'Value', 'Freq']
            lexica = lexica_1.merge(lexica_2, on=['Word'], how='inner')
            score += stats.pearsonr(lexica['Value_x'], lexica['Value_y'])[0]
        print(score/8)

Univariant FFN
0.2964588428418767
Univariant SVM
0.27284928014890564
Univariant LSTM
0.4525486486366554
Univariant Roberta_Mask
0.11574380832305828
Univariant Roberta_Partition
0.37027570925966613
Univariant DistilBERT_Mask
0.12660129861542443
Univariant DistilBERT_Partition
0.42421546731012194
FFN SVM
0.8770304803664702
FFN LSTM
0.2692479381963058
FFN Roberta_Mask
0.1704436100420134
FFN Roberta_Partition
0.23436658323758802
FFN DistilBERT_Mask
0.20710077091659487
FFN DistilBERT_Partition
0.20705238352376343
SVM LSTM
0.2631514467765863
SVM Roberta_Mask
0.176272999999214
SVM Roberta_Partition
0.23693712833351757
SVM DistilBERT_Mask
0.21603801238850442
SVM DistilBERT_Partition
0.20853987909078398
LSTM Roberta_Mask
0.15178916665675568
LSTM Roberta_Partition
0.2932418469564974
LSTM DistilBERT_Mask
0.17615897993008464
LSTM DistilBERT_Partition
0.2807025686242472
Roberta_Mask Roberta_Partition
0.3297848867341212
Roberta_Mask DistilBERT_Mask
0.3191512742015382
Roberta_Mask DistilBERT_Partitio